# Get Line of stations from Renfe data

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys

sys.path.append('..')

In [3]:
import datetime

from src.robin.scraping.renfe.entities import RenfeScraper

scraper = RenfeScraper(stations_csv_path='../data/renfe/renfe_stations.csv')

for station_id, station_name in scraper.available_stations.items():
    print(f'{station_id}: {station_name}')

?: Estaciones de Origen
31412: A Coruña
94707: Abrantes
60911: Alicante / Alacant
60600: Albacete
06008: Alcantarilla-Los Romanos
60400: Alcázar de San Juan
55020: Algeciras
56312: Almería
99003: Altet Bus
99115: Aguadulce Bus
87912: Aix En Provence
99114: Andorra-Bus
ANTEQ: Antequera (TODAS)
87814: Avignon
10400: Avila
37606: Badajoz
BARCE: Barcelona (TODAS)
87078: Beziers
65318: Benicassim
BILBA: Bilbao (TODAS)
54400: Bobadilla
11014: Burgos Rosa Manzano
35400: Cáceres
51405: Cádiz
70600: Calatayud
50417: Campus Rabanales
61307: Cartagena
65300: Castellón /Castelló
37200: Ciudad Real
50500: Córdoba
CUENC: Cuenca (TODAS)
92201: Denia-Bus
60905: Elda-Petrer
03410: Elche AV/Elx AV
94428: Entroncamento
92157: Estepona Bus
21010: Ferrol
79309: Figueres
79333: Figueres Bus
04307: Figueres Vilafant
69110: Gandía
GIJON: Gijón
79300: Girona
05000: Granada
GUADA: Guadalajara (TODAS)
43019: Huelva
74200: Huesca
IRUN-: Irun-Hendaya (TODAS)
80100: Pamplona/Iruña
99103: Jaca-Bus
03100: Jaén
64100:

In [4]:
scraper.stations_df

stop_id                       stop_name renfe_id   stop_lat  stop_lon
0    00000                         Unknown    00000   0.000000  0.000000
1    31412                       A Corunya    31412  43.352761 -8.409755
2    60911                Alicante/alacant    60911  38.344450 -0.495053
3    60600             Albacete-Los Llanos    60600  38.999384 -1.848450
4    60400             Alcazar de San Juan    60400  39.395628 -3.205744
..     ...                             ...      ...        ...       ...
92   13200  Bilbao-Abando Indalecio Prieto    BILBA  43.259609 -2.929150
93   66100                          Cuenca    CUENC  40.067340 -2.136471
94   15410                           Gijon    GIJON  43.535175 -5.698318
95   70200                     Guadalajara    GUADA  40.644103 -3.182230
96   78200                       Tardienta    78200  41.975751 -0.538314

[97 rows x 5 columns]

In [5]:
import pandas as pd 

trips = ['60000',  # MADRID
         '71801',  # BARCELONA
         '74200',  # HUESCA
         '15100',  # LEÓN
         '11014',  # BURGOS
         '31412',  # A CORUÑA
         '23004',  # PONTEVEDRA
         '51405',  # CÁDIZ
         '54413',  # MÁLAGA
         '05000',  # GRANADA
         '65300',  # CASTELLÓN
         '60911']  # ALICANTE

master = trips[0]
destinations = trips[1:]

df_trips, df_stops = pd.DataFrame(), pd.DataFrame()

for destination in destinations:
    for i in range(2):
        trip = None
        if i == 0:
            print(f'From {master} to {destination}')
            trip = (master, destination)
        else:
            print(f'From {destination} to {master}')
            trip = (destination, master)
        
        origin_id = scraper.get_renfe_station_id(trip[0])
        destination_id = scraper.get_renfe_station_id(trip[1])
    
        date = datetime.date(day=1, month=4, year=2024)
    
        buffer_df_trips, buffer_df_stops = scraper.scrape_trips(origin_id=origin_id, 
                                                                destination_id=destination_id, 
                                                                init_date=date,
                                                                range_days=30)
        
        df_trips = pd.concat([df_trips, buffer_df_trips], ignore_index=True)
        df_stops = pd.concat([df_stops, buffer_df_stops], ignore_index=True)
        print(f"######## ROWS OF DF_TRIPS: {df_trips.shape[0]} ############")
        
print(df_stops.head())

From 60000 to 71801
Date:  2024-04-01
Search url:  https://horarios.renfe.com/HIRRenfeWeb/buscar.do?O=MADRI&D=BARCE&AF=2024&MF=04&DF=01&SF=1&ID=s
  trip_id train_type                                           schedule  \
0   06301       AVLO             {'60000': (0, 0), '71801': (150, 150)}   
1   03063        AVE  {'60000': (0, 0), '70600': (55, 56), '04040': ...   
2   03073        AVE  {'60000': (0, 0), '70200': (23, 24), '04040': ...   
3   03083        AVE  {'60000': (0, 0), '04040': (75, 76), '71801': ...   
4   03091        AVE             {'60000': (0, 0), '71801': (150, 150)}   

            departure  duration                                   price  
0 2024-04-01 06:15:00       150                        {'Turist': 59.0}  
1 2024-04-01 06:30:00       170  {'Turist': 113.7, 'TuristaPlu': 151.2}  
2 2024-04-01 07:30:00       192  {'Turist': 113.7, 'TuristaPlu': 151.2}  
3 2024-04-01 08:30:00       165  {'Turist': 113.7, 'TuristaPlu': 151.2}  
4 2024-04-01 09:00:00       150  

/Users/uclm/PycharmProjects/robin/src/robin/scraping/renfe/entities.py:801: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_trips = pd.concat([df_trips, new_df_trips], ignore_index=True)


  trip_id train_type                                           schedule  \
0   05080        AVE  {'11014': (0, 0), '10600': (36, 38), '60000': ...   
1   04186      ALVIA  {'13200': (0, 0), '00000': (22, 23), '11200': ...   
2   04366      ALVIA  {'13200': (0, 0), '00000': (22, 23), '11200': ...   

            departure  duration                                 price  
0 2024-04-15 06:40:00        93  {'Turist': 61.1, 'TuristaPlu': 81.2}  
1 2024-04-15 09:34:00       114  {'Turist': 53.8, 'TuristaPlu': 71.6}  
2 2024-04-15 20:08:00       115  {'Turist': 53.8, 'TuristaPlu': 71.6}  
Date:  2024-04-16
Search url:  https://horarios.renfe.com/HIRRenfeWeb/buscar.do?O=11014&D=MADRI&AF=2024&MF=04&DF=16&SF=2&ID=s
  trip_id train_type                                           schedule  \
0   05080        AVE  {'11014': (0, 0), '10600': (36, 38), '60000': ...   
1   04186      ALVIA  {'13200': (0, 0), '00000': (22, 23), '11200': ...   
2   04366      ALVIA  {'13200': (0, 0), '00000': (22, 23), '

/Users/uclm/PycharmProjects/robin/src/robin/scraping/renfe/entities.py:801: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_trips = pd.concat([df_trips, new_df_trips], ignore_index=True)


  trip_id train_type                                           schedule  \
0   05080        AVE  {'11014': (0, 0), '10600': (36, 38), '60000': ...   
1   04186      ALVIA  {'13200': (0, 0), '00000': (22, 23), '11200': ...   
2   04366      ALVIA  {'13200': (0, 0), '00000': (22, 23), '11200': ...   

            departure  duration                                 price  
0 2024-04-22 06:40:00        93  {'Turist': 61.1, 'TuristaPlu': 81.2}  
1 2024-04-22 09:34:00       114  {'Turist': 53.8, 'TuristaPlu': 71.6}  
2 2024-04-22 20:08:00       115  {'Turist': 53.8, 'TuristaPlu': 71.6}  
Date:  2024-04-23
Search url:  https://horarios.renfe.com/HIRRenfeWeb/buscar.do?O=11014&D=MADRI&AF=2024&MF=04&DF=23&SF=2&ID=s
  trip_id train_type                                           schedule  \
0   05080        AVE  {'11014': (0, 0), '10600': (36, 38), '60000': ...   
1   04186      ALVIA  {'13200': (0, 0), '00000': (22, 23), '11200': ...   
2   04366      ALVIA  {'13200': (0, 0), '00000': (22, 23), '

/Users/uclm/PycharmProjects/robin/src/robin/scraping/renfe/entities.py:801: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_trips = pd.concat([df_trips, new_df_trips], ignore_index=True)


  trip_id train_type                                           schedule  \
0   05080        AVE  {'11014': (0, 0), '10600': (36, 38), '60000': ...   
1   04186      ALVIA  {'13200': (0, 0), '00000': (22, 23), '11200': ...   
2   04366      ALVIA  {'13200': (0, 0), '00000': (22, 23), '11200': ...   

            departure  duration                                 price  
0 2024-04-29 06:40:00        93  {'Turist': 61.1, 'TuristaPlu': 81.2}  
1 2024-04-29 09:34:00       114  {'Turist': 53.8, 'TuristaPlu': 71.6}  
2 2024-04-29 20:08:00       115  {'Turist': 53.8, 'TuristaPlu': 71.6}  
Date:  2024-04-30
Search url:  https://horarios.renfe.com/HIRRenfeWeb/buscar.do?O=11014&D=MADRI&AF=2024&MF=04&DF=30&SF=2&ID=s
  trip_id train_type                                           schedule  \
0   05080        AVE  {'11014': (0, 0), '10600': (36, 38), '60000': ...   
1   04186      ALVIA  {'13200': (0, 0), '00000': (22, 23), '11200': ...   
2   04366      ALVIA  {'13200': (0, 0), '00000': (22, 23), '

In [6]:
df_stops.head()

service_id stop_id  arrival  departure
0  06301_01-04-2024-06.15   60000        0          0
1  06301_01-04-2024-06.15   71801      150        150
2  03063_01-04-2024-06.30   60000        0          0
3  03063_01-04-2024-06.30   70600       55         56
4  03063_01-04-2024-06.30   04040       81         82

In [7]:
# Save dataframe to csv
df_stops.to_csv('stops_HSR_Spain_April_24.csv', index=False)

In [8]:
# Save dataframe to csv
df_trips.to_csv('trips_HSR_Spain_April_24.csv', index=False)

In [9]:
import pandas as pd

dtypes = {'stop_id': str}
df_stops = pd.read_csv('stops_HSR_Spain_April_24.csv', dtype=dtypes)
df_stops.head()

service_id stop_id  arrival  departure
0  06301_01-04-2024-06.15   60000        0          0
1  06301_01-04-2024-06.15   71801      150        150
2  03063_01-04-2024-06.30   60000        0          0
3  03063_01-04-2024-06.30   70600       55         56
4  03063_01-04-2024-06.30   04040       81         82

In [10]:
# Len of dataframe
print(len(df_stops))

19007


In [12]:
print(set(df_stops['stop_id']))

{'02003', '70200', '51400', '51300', '51003', '74200', '14100', '37300', '37200', '54413', '30200', '65000', '60400', '62002', '04307', '60600', '10600', '02002', '03213', '04040', '61200', '00000', '60000', '79300', '70101', '78200', '37704', '11200', '22100', '03410', '70600', '03309', '71801', '05000', '12100', '71500', '13200', '78400', '14223', '11014'}


In [13]:
print(set(scraper.stations_df['stop_id']))

{'51400', '20309', '51003', '92102', '20300', '15211', '60400', '15100', '62002', '60600', '50417', '10600', 'GUADA', '40008', 'TARRA', '61200', '43019', '79300', '23004', 'BILBA', '11200', 'MEDIN', '67200', 'S FER', '78400', '61307', '50300', '14100', '37300', '37200', '55020', '60902', '64100', '11208', 'VIGO-', 'IRUN-', '50102', '35206', '78301', '10400', '22100', '03410', '35400', '03100', '31412', 'CUENC', '11014', '51300', '54400', 'SANTA', '37500', 'MADRI', 'PUEBL', '04307', 'ANTEQ', '03213', '21010', '20200', '06006', '40100', '78200', '70600', '60905', 'REDON', '31400', '51405', '03309', 'GIJON', '65318', '05000', 'SEGOV', 'ZARAG', '60911', '65304', '30200', '23021', 'SALAM', '56312', '74200', 'PTE G', '54413', 'VALEN', '50500', 'BARCE', '82100', '37606', '15009', '00000', '81202', '37704', '71400', '65300', '79309', '81100', '11511', '69110', '79315'}


In [15]:
# Get set of stations which are NOT in the column renfe_id of scraper.stations_df
print(set(df_stops['stop_id']) - set(scraper.stations_df['stop_id']))

set()


In [8]:
# remove rows in wich stop_id is not in the the column renfe_id of scraper.stations_df
df_stops_clean = df_stops[df_stops['stop_id'].isin(scraper.stations_df['renfe_id'])]
df_stops_clean.reset_index(drop=True, inplace=True)
print(df_stops_clean)

                   service_id stop_id  arrival  departure
0      03063_01-03-2024-06.30   70600       55         56
1      03073_01-03-2024-07.30   78400      125        127
2      03093_01-03-2024-09.30   78400      119        121
3      03093_01-03-2024-09.30   79300      233        235
4      03093_01-03-2024-09.30   04307      250        250
...                       ...     ...      ...        ...
13673  05903_31-03-2024-20.01   03309       81         83
13674  05903_31-03-2024-20.01   60600      115        117
13675  05217_31-03-2024-21.04   00000        0          0
13676  05217_31-03-2024-21.04   03309       19         21
13677  05217_31-03-2024-21.04   60600       53         55

[13678 rows x 4 columns]


In [10]:
# save dataframe to csv
df_stops_clean.to_csv('stops_HSR_Spain_March_24_clean.csv', index=False)

In [15]:
result_dict = {}

grouped = df_stops.groupby("service_id")

for name, group in grouped:
    sub_dict = {}
    for index, row in group.iterrows():
        sub_dict[row['stop_id']] = (row['arrival'], row['departure'])
    result_dict[name] = sub_dict

print(result_dict)

{'03063_01-02-2024-06.30': {'60000': (0, 0), '70600': (55, 56), '04040': (81, 82), '71801': (170, 170)}, '03063_02-02-2024-06.30': {'60000': (0, 0), '70600': (55, 56), '04040': (81, 82), '71801': (170, 170)}, '03063_05-02-2024-06.30': {'60000': (0, 0), '70600': (55, 56), '04040': (81, 82), '71801': (170, 170)}, '03063_06-02-2024-06.30': {'60000': (0, 0), '70600': (55, 56), '04040': (81, 82), '71801': (170, 170)}, '03063_07-02-2024-06.30': {'60000': (0, 0), '70600': (55, 56), '04040': (81, 82), '71801': (170, 170)}, '03063_08-02-2024-06.30': {'60000': (0, 0), '70600': (55, 56), '04040': (81, 82), '71801': (170, 170)}, '03063_09-02-2024-06.30': {'60000': (0, 0), '70600': (55, 56), '04040': (81, 82), '71801': (170, 170)}, '03063_12-02-2024-06.30': {'60000': (0, 0), '70600': (55, 56), '04040': (81, 82), '71801': (170, 170)}, '03063_13-02-2024-06.30': {'60000': (0, 0), '70600': (55, 56), '04040': (81, 82), '71801': (170, 170)}, '03063_14-02-2024-06.30': {'60000': (0, 0), '70600': (55, 56), 

In [17]:
trips = list(set(tuple(service.keys())for service in result_dict.values()))
# print(trips)

In [18]:
# Initialize line with max length trip
line_stations = list(trips.pop(trips.index(max(trips, key=len))))

# Complete corridor with other stops that are not in the initial defined corridor
for trip in trips:
    for i, station in enumerate(trip):
        if station not in line_stations:
            # If station is the last one, append it to the end of the corridor
            if i == len(trip) - 1:
                line_stations.append(station)
            else:
                # If station is not the last one, insert it in the corridor before the next station
                index = line_stations.index(trip[i + 1])
                line_stations.insert(index, station)

print(line_stations)

['60000', '70200', '70600', '04040', '78200', '74200']


In [11]:
mapped_names = scraper.stations_df.set_index('stop_id')['stop_name'].to_dict()
line_stations_names = list(map(mapped_names.get, line_stations))

print(line_stations_names)

['Madrid-Puerta de Atocha', 'Guadalajara', 'Calatayud', 'Zaragoza-Delicias', 'Tarragona', 'Lleida', 'Barcelona-Sants', 'Girona', 'Figueres Vilafant', 'Perpignan', 'Narbonne', 'Beziers', 'Montpellier Saint Roch', 'Nimes', 'Avignon TGV', 'Aix En Provence TGV', 'Marseille St Charles']


In [ ]:
# ['Madrid-Puerta de Atocha', 'Guadalajara', 'Calatayud', 'Zaragoza-Delicias', 'Tardienta', 'Huesca']

['Madrid-Puerta de Atocha', 'Guadalajara', 'Calatayud', 'Zaragoza-Delicias', [['Tarragona', 'Lleida', 'Barcelona-Sants', 'Girona', 'Figueres Vilafant'], ['Tardienta', 'Huesca']]]

In [ ]:
['60000', '70200', '70600', '04040'], [['71500', '78400', '71801', '79300', '04307'], ['78200', '74200']]

In [19]:
df_stations = scraper.stations_df.copy()

print(df_stations.head())

  stop_id            stop_name renfe_id   stop_lat  stop_lon
0   00000              Unknown    00000   0.000000  0.000000
1   31412            A Corunya    31412  43.352761 -8.409755
2   60911     Alicante/alacant    60911  38.344450 -0.495053
3   60600  Albacete-Los Llanos    60600  38.999384 -1.848450
4   60400  Alcazar de San Juan    60400  39.395628 -3.205744


In [22]:
estaciones_noreste = ['60000', '70200', '70600', '04040', '71500', '78400', '71801', '79300', '04307', '78200', '74200']

df_stations = df_stations[df_stations['stop_id'].isin(estaciones_noreste)]
df_stations.drop(columns=['renfe_id'], inplace=True)
df_stations.reset_index(drop=True, inplace=True)
print(df_stations)

   stop_id                stop_name   stop_lat  stop_lon
0    70600                Calatayud  41.346692 -1.638680
1    04307        Figueres Vilafant  42.264771  2.943547
2    79300                   Girona  41.979303  2.817006
3    74200                   Huesca  42.133594 -0.409745
4    78400                   Lleida  41.620696  0.632669
5    60000  Madrid-Puerta de Atocha  40.406442 -3.690886
6    71500                Tarragona  41.111624  1.253214
7    04040        Zaragoza-Delicias  41.658649 -0.911615
8    71801          Barcelona-Sants  41.379220  2.140624
9    70200              Guadalajara  40.644103 -3.182230
10   78200                Tardienta  41.975751 -0.538314


In [23]:
# save dataframe to csv
df_stations.to_csv('estaciones_corredor_noreste.csv', index=False)